In [1]:
from vtk import *
import itk

In [10]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

#base_image = itk.imread("./data/BRATS_HG0015_T1C.mha")
base_image = itk.ImageFileReader(FileName="./data/BRATS_HG0015_T1C.mha")

rescaled = itk.RescaleIntensityImageFilter.New(
    Input=base_image,
    OutputMinimum=0,
    OutputMaximum=255
)

binary_im = itk.ThresholdImageFilter.New(
    Input=rescaled,
    Lower=102,
)

connected_components = itk.ConnectedComponentImageFilter.New(
    Input=binary_im,
)

filtered_cc = itk.LabelShapeKeepNObjectsImageFilter.New(
    Input=connected_components,
    BackgroundValue=0,
    NumberOfObjects=2,
    Attribute="NumberOfPixels"
)

# im = itk.LabelShapeKeepNObjectsImageFilter.New(
#     Input=im,
#     BackgroundValue=0,
#     NumberOfObjects=5,
#     Attribute="Roundness"
# )
# im.Update()

#im = itk.ThresholdImageFilter.New(
#    Input=im,
#    Lower=1,
#    OutsideValue=0,
#)
#im.Update()

mask = itk.NotImageFilter(Input=filtered_cc)
mask = itk.NotImageFilter(Input=mask)

result_im = itk.RescaleIntensityImageFilter.New(
    Input=mask,
    OutputMinimum=0,
    OutputMaximum=255
)

writer = itk.ImageFileWriter(FileName="./data/mask.mha")
writer.Update()

array = itk.GetArrayFromImage(result_im)

RuntimeError: /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageFileWriter.hxx:90:
itk::ERROR: itk::ERROR: ImageFileWriter(0x7efdf0041000): No input to writer!

In [9]:
np.unique(array, return_counts=True)

(array([  0, 255], dtype=int16), array([6026929,   55631]))

In [4]:
def _check_valid_arg(val, name, available):
    if val not in available:
        raise f"{name}='{val}' is not a valid arg. Valid values are: {available}"
        
def load_volume(reader, reader_mask, min_intensity=0, intensity_coeff=0.5, color=(1.,1.,1.), opacity_window=4096, render_with="gl", interpolation="linear"):
    _check_valid_arg(render_with, "render_with", {'gl', 'gpu', 'cpu'})
    _check_valid_arg(interpolation, "interpolation", {'linear', 'nearest'})
    
    reader.Update()
    reader_mask.Update()

    if render_with == "gl":
        mapper = vtkOpenGLGPUVolumeRayCastMapper()
    elif render_with == 'gpu':
        mapper = vtkGPUVolumeRayCastMapper() 
    elif render_with == 'cpu':
        mapper = vtkFixedPointVolumeRayCastMapper()
    else:
        raise "unexpected"
        
    mapper.SetInputConnection(reader.GetOutputPort())
    mapper.SetSampleDistance(0.5)
#     mapper.SetMaskTypeToBinary()
    mapper.SetMaskInput(reader_mask.GetOutput())
    
    mapper.SetBlendModeToMaximumIntensity()

    data_min, data_max = reader.GetOutput().GetScalarRange()
    
    seg_min, seg_max = min_intensity, min_intensity + (1-intensity_coeff) * (data_max-min_intensity)
    fct_color = vtkColorTransferFunction()
    fct_color.AddRGBSegment(seg_min, *(0., 0., 0.),
                            seg_max, *color)
    
    fct_opacity = vtkPiecewiseFunction()
    fct_opacity.AddSegment(seg_min, 0.,
                           seg_max, 1.)
    
    props = vtkVolumeProperty()
    props.SetIndependentComponents(True)
    props.SetColor(fct_color)
    props.SetScalarOpacity(fct_opacity)

    if interpolation == "linear":
        props.SetInterpolationTypeToLinear()
    elif interpolation == 'nearest':
        props.SetInterpolationTypeToNearest()
    else:
        raise "unexpected"

    volume = vtkVolume()
    volume.SetMapper(mapper)
    volume.SetProperty(props)
    
    return volume, props


reader_brain = vtkMetaImageReader()
reader_brain.SetFileName("./data/BRATS_HG0015_T1C.mha")
reader_mask = vtkImageReader()
reader_mask.SetFileName("./data/mask.mha")
volume, _ = load_volume(reader_brain, reader_mask, intensity_coeff=0.4)

ren = vtkRenderer()
ren.AddVolume(volume)

renWin = vtkRenderWindow()
renWin.AddRenderer(ren)

iren = vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
iren.Initialize()

slider = vtkSliderRepresentation2D()
slider.SetMinimumValue(1)
slider.SetMaximumValue(10)
slider.SetValue(1)
slider.SetTitleText("Number of whaterver")
slider.ShowSliderLabelOn()
slider.GetPoint1Coordinate().SetCoordinateSystemToDisplay();
slider.GetPoint1Coordinate().SetValue(300, 200);
slider.GetPoint2Coordinate().SetCoordinateSystemToDisplay();
slider.GetPoint2Coordinate().SetValue(600, 200);

sliderWidget = vtkSliderWidget()
sliderWidget.SetInteractor(iren);
sliderWidget.SetRepresentation(slider);
sliderWidget.SetAnimationModeToAnimate();
sliderWidget.EnabledOn();

def OnClose(interactor, event):
    interactor.GetRenderWindow().Finalize()
    interactor.TerminateApp()

iren.AddObserver('ExitEvent', OnClose)
renWin.Render()
iren.Start()